
### Question 1: [IPO] Withdrawn IPOs by Company Type

**What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**

From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn](https://stockanalysis.com/ipos/withdrawn/)), collect and process the data to find out which company type saw the most withdrawn IPO value.

#### Steps:
1. Use `pandas.read_html()` with the URL above to load the IPO withdrawal table into a DataFrame. 
   *It is a similar process to Code Snippet 1 discussed at the livestream.*    You should get **99 entries**. 
2. Create a new column called `Company Class`, categorizing company names based on patterns like:
   - “Acquisition Corp” or “Acquisition Corporation” → `Acq.Corp`
   - “Inc” or “Incorporated” → `Inc`
   - “Group” → `Group`
   - “Ltd” or “Limited” → `Limited`
   - “Holdings” → `Holdings`
   - Others → `Other`

  *  Order: Please follow the listed order of classes and assign the first matched value (e.g., for 'shenni holdings limited', you assign the 'Limited' class).

  * Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.

3. Define a new field `Avg. price` by parsing the `Price Range` field (create a function and apply it to the `Price Range` column). Examples:
   - '$8.00-$10.00' → `9.0`  
   - '$5.00' → `5.0`  
   - '-' → `None`
4. Convert `Shares Offered` to numeric, clean missing or invalid values.
5. Create a new column:  
   `Withdrawn Value = Shares Offered * Avg Price` (**71 non-null values**)
6. Group by `Company Class` and calculate total withdrawn value.
7. **Answer**: Which class had the highest **total** value of withdrawals?

In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import requests

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt
import pandas as pd
import requests
from io import StringIO

# Function accepts link and return a DataFrame 


def get_ipos_df(link: str) -> pd.DataFrame:    
	"""
	Fetch IPO data for the given link and return a DataFrame 
	"""
	url = link
	headers = {
		'User-Agent': (
			'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
			'AppleWebKit/537.36 (KHTML, like Gecko) '
			'Chrome/58.0.3029.110 Safari/537.3'
		)
	}

	try:
		response = requests.get(url, headers=headers, timeout=10)
		response.raise_for_status()
		html_io = StringIO(response.text)
		tables = pd.read_html(html_io)

		if not tables:
			raise ValueError(f"No tables found.")
		return tables[0]

	except requests.exceptions.RequestException as e:
		print(f"Request failed: {e}")
	except ValueError as ve:
		print(f"Data error: {ve}")
	except Exception as ex:
		print(f"Unexpected error: {ex}")

	return pd.DataFrame()

C:\Users\adity\AppData\Local\Temp\ipykernel_5832\1108122396.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
ipos_wd = get_ipos_df("https://stockanalysis.com/ipos/withdrawn/")
ipos_wd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          101 non-null    object
 1   Company Name    101 non-null    object
 2   Price Range     101 non-null    object
 3   Shares Offered  101 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [3]:

def categorize_company_name(name: str) -> str:
	"""
	Categorize company names based on specific patterns.
	"""
	name = name.lower()
	if 'acquisition corp' in name or 'acquisition corporation' in name:
		return 'Acq.Corp'
	elif 'incorporated' in name or 'inc' in name:
		return 'Inc'
	elif 'group' in name:
		return 'Group'
	elif 'limited' in name or 'ltd' in name:
		return 'Ltd'
	elif 'holdings' in name:
		return 'Holdings'
	else:
		return 'Other'
# Apply the categorization function to the 'Company Name' column
ipos_wd['Company Class'] = ipos_wd['Company Name'].apply(categorize_company_name)
ipos_wd.info()
ipos_wd.head(10)
# This code does not consider the order and there are issues with holdings and other types 
# ipos_wd['Lower Cname'] = ipos_wd['Company Name'].str.lower()
# ipos_wd['Company Class'] = ipos_wd['Lower Cname'].str.extract(r'(acquisition corp|acquisition corporation|inc|incorporated|group|ltd|limited|holdings)', expand=False)
# ipos_wd['Company Class'] = ipos_wd['Company Class'].fillna('Other')
# ipos_wd['Company Class'] = ipos_wd['Company Class'].replace({'acquisition Corp': 'Acq.Corp','acquisition corporation':'Acq. Corp','incorporated': 'Inc','inc':'Inc','group':'Group','limited':'Ltd','ltd':'Ltd','holdings': 'Holdings'})

non_null_so = ipos_wd['Shares Offered'].notnull().sum()
non_null_pr = ipos_wd['Price Range'].notnull().sum()

print(f"Number of non-null values in Shares Offered: {non_null_so}")
print(f"Number of non-null values in Price Range: {non_null_pr}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          101 non-null    object
 1   Company Name    101 non-null    object
 2   Price Range     101 non-null    object
 3   Shares Offered  101 non-null    object
 4   Company Class   101 non-null    object
dtypes: object(5)
memory usage: 4.1+ KB
Number of non-null values in Shares Offered: 101
Number of non-null values in Price Range: 101


In [4]:

def parse_price_range(price_range: str) -> float:
	"""
	Parse the price range string and return the average price.
	"""
	if price_range == '-':
		return np.nan
	elif '-' in price_range:
		try:
			low, high = price_range.split('-')
			low = float(low.replace('$', '').strip())
			high = float(high.replace('$', '').strip())
			return (low + high) / 2
		except ValueError:
			return None
	else:
		try:
			return float(price_range.replace('$', '').strip())
		except ValueError:
			return None

ipos_wd['Avg. Price'] = ipos_wd['Price Range'].apply(parse_price_range)
ipos_wd.info()
ipos_wd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Symbol          101 non-null    object 
 1   Company Name    101 non-null    object 
 2   Price Range     101 non-null    object 
 3   Shares Offered  101 non-null    object 
 4   Company Class   101 non-null    object 
 5   Avg. Price      73 non-null     float64
dtypes: float64(1), object(5)
memory usage: 4.9+ KB


,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. Price
0,EIL,E I L Holdings Limited,-,-,Ltd,NaN
1,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
2,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc,3.5
3,AURN,"Aurion Biotech, Inc.",-,-,Inc,NaN
4,ROTR,"PHI Group, Inc.",-,-,Inc,NaN
...,...,...,...,...,...,...
96,FHP,"Freehold Properties, Inc.",-,-,Inc,NaN
97,CHO,Chobani Inc.,-,-,Inc,NaN
98,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc,19.5
99,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc,NaN


In [5]:

def parse_shares_offered(shares: str) -> float:
	"""
	Parse the shares offered string and return the number of shares as an integer.
	"""
	if shares == '-':
		return None
	try:
		# Remove any non-numeric characters except for commas
		cleaned_shares = shares.replace(',', '').replace(' ', '')
		return float(cleaned_shares)
	except ValueError:
		return None

ipos_wd['Shares Offered'] = ipos_wd['Shares Offered'].apply(parse_shares_offered)
ipos_wd.info()
ipos_wd.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Symbol          101 non-null    object 
 1   Company Name    101 non-null    object 
 2   Price Range     101 non-null    object 
 3   Shares Offered  72 non-null     float64
 4   Company Class   101 non-null    object 
 5   Avg. Price      73 non-null     float64
dtypes: float64(2), object(4)
memory usage: 4.9+ KB


,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. Price
0,EIL,E I L Holdings Limited,-,NaN,Ltd,NaN
1,ODTX,"Odyssey Therapeutics, Inc.",-,NaN,Inc,NaN
2,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000.0,Inc,3.5
3,AURN,"Aurion Biotech, Inc.",-,NaN,Inc,NaN
4,ROTR,"PHI Group, Inc.",-,NaN,Inc,NaN
5,ONE,One Power Company,-,NaN,Other,NaN
6,HPOT,The Great Restaurant Development Holdings Limited,$4.00 - $6.00,1400000.0,Ltd,5.0
7,CABR,"Caring Brands, Inc.",$4.00,750000.0,Inc,4.0
8,SQVI,"Sequoia Vaccines, Inc.",$8.00 - $10.00,2775000.0,Inc,9.0
9,SNI,Shenni Holdings Limited,$4.00 - $6.00,3000000.0,Ltd,5.0


In [6]:
ipos_wd['Withdrawn Value'] = ipos_wd['Shares Offered'] * ipos_wd['Avg. Price']
ipos_wd.head(100)
withdrawn_value_non_null = ipos_wd['Withdrawn Value'].notnull().sum()
print(f"Number of non-null values in Withdrawn Value: {withdrawn_value_non_null}")

Number of non-null values in Withdrawn Value: 71


In [7]:
withdrawn_value_by_class = ipos_wd.groupby('Company Class')['Withdrawn Value'].sum().reset_index()
withdrawn_value_by_class['Withdrawn Value'] = withdrawn_value_by_class['Withdrawn Value'] / 1_000_000  # Convert to millions
withdrawn_value_by_class = withdrawn_value_by_class.sort_values(by='Withdrawn Value', ascending=False)
print(withdrawn_value_by_class)

  Company Class  Withdrawn Value
0      Acq.Corp      4021.000000
3           Inc      2257.164205
5         Other       767.919999
4           Ltd       549.734585
2      Holdings        75.000000
1         Group        33.787500


### Question 2:   [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)


**What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?**

The goal is to replicate the large-scale `yfinance` OHLCV data download and perform basic financial calculations on IPO stocks.


#### Steps:

1. Using the same approach as in Question 1, download the IPOs in 2024 from:  
   [https://stockanalysis.com/ipos/2024/](https://stockanalysis.com/ipos/2024/)  
   Filter to keep only those IPOs **before 1 June 2024** (first 5 months of 2024).  
   ➤ You should have **75 tickers**.

2.  Use **Code Snippet 7** to download daily stock data for those tickers (via `yfinance`).  
   Make sure you understand how `growth_1d` ... `growth_365d`, and volatility columns are defined.  
   Define a new column `growth_252d` representing growth after **252 trading days** (~1 year), in addition to any other growth periods you already track.


3. Calculate the Sharpe ratio assuming a risk-free rate of **4.5%**:

   ```python
   stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']
   ```

   ⚠️ **IMPORTANT** Please use the original version of annualized volatility calculation (it was later corrected to another formula):
   ```python
   stocks_df['volatility'] =   stocks_df['Close'].rolling(30).std() * np.sqrt(252)
   ```
4. Filter the DataFrame to keep data only for the trading day:  
   **‘2025-06-06’**

   Compute descriptive statistics (e.g., `.describe()`) for these columns:  
   - `growth_252d`  
   - `Sharpe`

   You should observe:  
   - `growth_252d` is defined for **71 out of 75 stocks** (some IPOs are too recent or data starts later).  
   - Median `growth_252d` is approximately **0.75** (indicating a 25% decline), while mean is about **1.15**, showing a bias towards high-growth companies pushing the average up.

5. **Answer:**  
   - What is the **median Sharpe ratio** for these 71 stocks?  
   - Note: Positive `Sharpe` means growth exceeding the risk-free rate of 4.5%.  
   - [Additional] Do you observe the **same top 10 companies** when sorting by `growth_252d` versus sorting by `Sharpe`?


In [8]:
# Using the same approach as in Question 1, download the IPOs in 2024 from:
# https://stockanalysis.com/ipos/2024/
# Filter to keep only those IPOs before 1 June 2024 (first 5 months of 2024).
# ➤ You should have 75 tickers.

ipos_2024 = get_ipos_df("https://stockanalysis.com/ipos/2024/")
ipos_2024.info()
# Filter to keep only those IPOs before 1 June 2024
ipos_2024['IPO Date'] = pd.to_datetime(ipos_2024['IPO Date'], errors='coerce')
ipos_2024 = ipos_2024[ipos_2024['IPO Date'] < pd.Timestamp('2024-06-01')]
# Remove rows with "-", " " in 'IPO Price'
ipos_2024 = ipos_2024[ipos_2024['IPO Price'].notnull() & (ipos_2024['IPO Price'] != '-') & (ipos_2024['IPO Price'] != ' ')]
# Check the number of tickers
print(f"Number of tickers in 2024 before June: {len(ipos_2024['Symbol'])}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      225 non-null    object
 1   Symbol        225 non-null    object
 2   Company Name  225 non-null    object
 3   IPO Price     225 non-null    object
 4   Current       225 non-null    object
 5   Return        225 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB
Number of tickers in 2024 before June: 75


In [9]:
stocks_df = pd.DataFrame(ipos_2024['Symbol'])

all_stocks_data = []

for ticker in ipos_2024['Symbol']:
    try:
        # Fetch the stock data for one ticker
        ticker_obj = yf.Ticker(ticker)
        hist_df = ticker_obj.history(period="max", interval="1d")

        if hist_df.empty:
            print(f"No data found for {ticker}, skipping.")
            continue

        # Add ticker symbol and date features
        hist_df['Ticker'] = ticker
        hist_df['Year'] = hist_df.index.year
        hist_df['Month'] = hist_df.index.month
        hist_df['Weekday'] = hist_df.index.weekday
        hist_df['Date'] = hist_df.index.date

        # Define a new column growth_252d representing growth after 252 trading days
        hist_df['growth_252d'] = hist_df['Close'] / hist_df['Close'].shift(252)

        # Calculate volatility using the specified formula
        hist_df['volatility'] = hist_df['Close'].rolling(30).std() * np.sqrt(252)

        # Calculate the Sharpe ratio assuming a risk-free rate of 4.5%
        # Note: The standard Sharpe Ratio uses returns (growth - 1) and volatility of returns.
        risk_free_rate = 0.045
        hist_df['Sharpe'] = (hist_df['growth_252d'] - risk_free_rate) / hist_df['volatility']

        all_stocks_data.append(hist_df)
        print(f"Successfully processed {ticker}")

    except Exception as e:
        print(f"Error fetching or processing data for {ticker}: {e}")

# Combine the list of DataFrames into a single DataFrame
if all_stocks_data:
    stocks_df = pd.concat(all_stocks_data)
    print("\nData processing complete. `stocks_df` contains all data.")
    # print(stocks_df.tail())
else:
    stocks_df = pd.DataFrame()
    print("\nNo data was fetched. `stocks_df` is empty.")

Successfully processed BOW
Successfully processed HDL
Successfully processed RFAI
Successfully processed JDZG
Successfully processed RAY
Successfully processed BTOC
Successfully processed ZK
Successfully processed GPAT
Successfully processed PAL
Successfully processed SVCO
Successfully processed NNE
Successfully processed CCIX
Successfully processed VIK
Successfully processed ZONE
Successfully processed LOAR
Successfully processed MRX
Successfully processed RBRK
Successfully processed NCI
Successfully processed MFI
Successfully processed YYGH
Successfully processed TRSG
Successfully processed CDTG
Successfully processed CTRI
Successfully processed IBTA
Successfully processed MTEN
Successfully processed TWG
Successfully processed ULS
Successfully processed PACS
Successfully processed MNDR
Successfully processed CTNM
Successfully processed MAMO
Successfully processed ZBAO
Successfully processed BOLD
Successfully processed MMA
Successfully processed UBXG
Successfully processed IBAC
Succes

In [10]:
stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 23621 entries, 2024-05-23 00:00:00-04:00 to 2025-06-26 00:00:00-04:00
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          23621 non-null  float64
 1   High          23621 non-null  float64
 2   Low           23621 non-null  float64
 3   Close         23621 non-null  float64
 4   Volume        23621 non-null  int64  
 5   Dividends     23621 non-null  float64
 6   Stock Splits  23621 non-null  float64
 7   Ticker        23621 non-null  object 
 8   Year          23621 non-null  int32  
 9   Month         23621 non-null  int32  
 10  Weekday       23621 non-null  int32  
 11  Date          23621 non-null  object 
 12  growth_252d   4787 non-null   float64
 13  volatility    21446 non-null  float64
 14  Sharpe        4787 non-null   float64
dtypes: float64(9), int32(3), int64(1), object(2)
memory usage: 2.6+ MB


In [11]:
# Filter the DataFrame to keep data only for the trading day:
# ‘2025-06-06’
trading_day = pd.Timestamp('2025-06-06')
filtered_stocks_df = stocks_df[stocks_df['Date'] == trading_day.date()]
filtered_stocks_df = filtered_stocks_df[['Ticker', 'growth_252d', 'Sharpe']]
filtered_stocks_df = filtered_stocks_df.dropna(subset=['growth_252d', 'Sharpe'])
print(len(filtered_stocks_df))

# Compute descriptive statistics (e.g., .describe()) for these columns: growth_252d Sharpe

stats = filtered_stocks_df[['growth_252d', 'Sharpe']].describe()
print("\nDescriptive statistics for growth_252d and Sharpe:")
print(stats)

71

Descriptive statistics for growth_252d and Sharpe:
       growth_252d     Sharpe
count    71.000000  71.000000
mean      1.152898   0.287253
std       1.406017   0.519513
min       0.024970  -0.079677
25%       0.293422   0.039684
50%       0.758065   0.080707
75%       1.362736   0.311507
max       8.097413   2.835668


### Question 3: [IPO] ‘Fixed Months Holding Strategy’

**What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?**  
(*Assume you buy at the close of the first trading day and sell after a fixed number of trading days.*)


---

#### Goal:
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

#### Steps:

1. **Start from the existing DataFrame** from Question 2 (75 tickers from IPOs in the first 5 months of 2024).  

   Add **12 new columns**:  
   `future_growth_1m`, `future_growth_2m`, ..., `future_growth_12m`  
   *(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)*  
   This logic is similar to `stocks_df['growth_future_30d']` from **Code Snippet 7**, but extended to longer timeframes.

2. **Determine the first trading day** (`min_date`) for each ticker.  
   This is the earliest date in the data for each stock.

3. **Join the data**:  
   Perform an **inner join** between the `min_date` DataFrame and the future growth data on both `ticker` and `date`.  
   ➤ You should end up with **75 records** (one per IPO) with all 12 `future_growth_...` fields populated.

4. **Compute descriptive statistics** for the resulting DataFrame:  
   Use `.describe()` or similar to analyze each of the 12 columns:  
   - `future_growth_1m`  
   - `future_growth_2m`  
   - ...  
   - `future_growth_12m`  

5. **Determine the best holding period**:  
   - Find the number of months **(1 to 12)** where the **average (mean)** future growth is **maximal**.  
   - This optimal month shows an uplift of **>1%** compared to all others.  
   - Still, the average return remains **less than 1** (i.e., expected return is less than doubling your investment).

In [12]:
# # Add 12 new columns:
# # future_growth_1m, future_growth_2m, ..., future_growth_12m
# # (Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)
# # This logic is similar to historyPrices['growth_future_30d'] from Code Snippet 7, but extended to longer timeframes.
# # stocks_df = stocks_df.dropna(subset=['growth_252d', 'Sharpe'])

# for i in range(1, 13):
# 	days = i * 21  # 21 trading days per month
# 	stocks_df[f'future_growth_{i}m'] = stocks_df['Close'] / stocks_df['Close'].shift(days)
# # remove dates from dataframe
# stocks_df = stocks_df.drop(columns=['Date'])
# # convert Ticker to string type
# stocks_df = stocks_df.reset_index()	
# # remove time from date
# stocks_df['Date'] = pd.to_datetime(stocks_df['Date']).dt.date
# stocks_df['Date'] = pd.to_datetime(stocks_df['Date'], errors='coerce')

# stocks_df.info()
# stocks_df.head(10)

In [13]:
# min_dates = stocks_df.groupby('Ticker').agg(
#     Date=('Date', 'min')  # Find the min of 'Date' and name the new column 'Date'
# ).reset_index()

# min_dates.info()

In [14]:
# # Join the data:
# # Perform an inner join between the min_date DataFrame and the future growth data on both ticker and date.
# ipo_first_day_growth = pd.merge(
#     left=min_dates,
#     right=stocks_df,
#     on=['Ticker', 'Date'], # The 'on' parameter can be used directly
#     how='inner'
# )


# # ➤ You should end up with 75 records (one per IPO) with all 12 future_growth_... fields populated.
# ipo_first_day_growth.info()

# ipo_first_day_growth.head(10)


In [15]:

# ipo_first_day_growth.describe(
# 	include=[np.number]  # Include only numeric columns
# ).filter(like='future_growth_')


# # Determine the best holding period:

# # Find the number of months (1 to 12) where the average (mean) future growth is maximal.
# # This optimal month shows an uplift of >1% compared to all others.
# # Still, the average return remains less than 1 (i.e., expected return is less than doubling your investment).

# optimal_months = ipo_first_day_growth.filter(like='future_growth_').mean().idxmax()
# optimal_months_value = ipo_first_day_growth[optimal_months].mean()
# print(f"The optimal month for future growth is: {optimal_months} with an average growth of {optimal_months_value:.4f}")


## Approach 2 : Reference taken from [here](https://github.com/rsyjswy23/stock-markets-analytics-zoomcamp/blob/main/cohorts/2025/HW2_solution.ipynb)


In [16]:
ALL_TICKERS = ipos_2024['Symbol'].values
ALL_TICKERS

array(['BOW', 'HDL', 'RFAI', 'JDZG', 'RAY', 'BTOC', 'ZK', 'GPAT', 'PAL',
       'SVCO', 'NNE', 'CCIX', 'VIK', 'ZONE', 'LOAR', 'MRX', 'RBRK', 'NCI',
       'MFI', 'YYGH', 'TRSG', 'CDTG', 'CTRI', 'IBTA', 'MTEN', 'TWG',
       'ULS', 'PACS', 'MNDR', 'CTNM', 'MAMO', 'ZBAO', 'BOLD', 'MMA',
       'UBXG', 'IBAC', 'AUNA', 'BKHA', 'LOBO', 'RDDT', 'ALAB', 'INTJ',
       'RYDE', 'LGCL', 'SMXT', 'VHAI', 'DYCQ', 'CHRO', 'UMAC', 'HLXB',
       'MGX', 'TBBB', 'TELO', 'KYTX', 'PMNT', 'AHR', 'LEGT', 'ANRO',
       'GUTS', 'AS', 'FBLG', 'AVBP', 'BTSG', 'HAO', 'CGON', 'YIBO', 'JL',
       'SUGP', 'JVSA', 'KSPI', 'CCTG', 'PSBD', 'SYNX', 'SDHC', 'ROMA'],
      dtype=object)

In [17]:
# # Step 1: Start from 75 tickers from IPOs in the first 5 months of 2024 from Q2. Add 12 new columns:future_growth_1m to 12m. Similar to Snippet 7, but extend to longer timeframes.

future_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
	print(i,ticker)

	# Work with stock prices
	ticker_obj = yf.Ticker(ticker)

	historyPrices = ticker_obj.history(
						period = "max",
						interval = "1d")

	# generate features for historical prices, and what we want to predict
	historyPrices['Ticker'] = ticker
	historyPrices['Date'] = historyPrices.index.date
	
	# historical returns
	for i in range(1,13):
		historyPrices[f'future_growth_{i}m'] = historyPrices['Close'].shift(-i*21) / historyPrices['Close']
	# sleep 1 sec between downloads - not to overload the API server
	time.sleep(1)


	if future_df.empty:
		future_df = historyPrices
	else:
		future_df = pd.concat([future_df, historyPrices], ignore_index=True)

0 BOW
1 HDL
2 RFAI
3 JDZG
4 RAY
5 BTOC
6 ZK
7 GPAT
8 PAL
9 SVCO
10 NNE
11 CCIX
12 VIK
13 ZONE
14 LOAR
15 MRX
16 RBRK
17 NCI
18 MFI
19 YYGH
20 TRSG
21 CDTG
22 CTRI
23 IBTA
24 MTEN
25 TWG
26 ULS
27 PACS
28 MNDR
29 CTNM
30 MAMO
31 ZBAO
32 BOLD
33 MMA
34 UBXG
35 IBAC
36 AUNA
37 BKHA
38 LOBO
39 RDDT
40 ALAB
41 INTJ
42 RYDE
43 LGCL
44 SMXT
45 VHAI
46 DYCQ
47 CHRO
48 UMAC
49 HLXB
50 MGX
51 TBBB
52 TELO
53 KYTX
54 PMNT
55 AHR
56 LEGT
57 ANRO
58 GUTS
59 AS
60 FBLG
61 AVBP
62 BTSG
63 HAO
64 CGON
65 YIBO
66 JL
67 SUGP
68 JVSA
69 KSPI
70 CCTG
71 PSBD
72 SYNX
73 SDHC
74 ROMA


In [18]:
# Step 2: Determine the first trading day (min_date) for each ticker. Earliest date in the data for each stock.
min_date = future_df.groupby('Ticker')['Date'].min()
min_date_df = pd.DataFrame(min_date)
min_date_df.head()


,Date
Ticker,
AHR,2024-02-07
ALAB,2024-03-20
ANRO,2024-02-02
AS,2024-02-01
AUNA,2024-03-22


In [19]:
# Step 3: Join the data: Perform an inner join between the min_date DataFrame and the future growth data on both ticker and date.
# need to reset idx to bring Ticker to columns
min_date_df_reset = min_date_df.reset_index()
merged = pd.merge(future_df, min_date_df_reset, on=['Ticker', 'Date'], how='inner')
merged

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Date,future_growth_1m,...,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,23.000000,24.270000,22.139999,23.799999,3335800,0.0,0.0,BOW,2024-05-23,1.024790,...,1.301681,1.167647,1.239076,1.461765,1.443698,1.357983,1.389076,1.722689,1.698319,1.533613
1,27.000000,30.000000,19.799999,22.290001,670000,0.0,0.0,HDL,2024-05-17,0.775236,...,0.673845,0.643786,0.720502,0.719605,1.255271,1.106326,1.030507,1.192014,0.907133,1.002602
2,10.000000,10.080000,10.000000,10.040000,1175800,0.0,0.0,RFAI,2024-07-05,1.002191,...,1.007968,1.014940,1.014940,1.017928,1.022908,1.030877,1.035857,1.040837,1.046813,NaN
3,38.400002,43.200001,28.760000,32.560001,295775,0.0,0.0,JDZG,2024-05-15,0.242998,...,0.122850,0.160442,0.272727,0.199017,0.139066,0.182555,0.170025,0.146437,0.121867,0.142506
4,5.050000,6.040000,4.020000,4.230000,2379300,0.0,0.0,RAY,2024-05-15,0.839243,...,0.368794,0.463357,0.397163,0.406619,0.395508,0.290780,0.297872,0.255319,0.602837,0.432624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,5.750000,9.833000,5.030000,7.700000,5181000,0.0,0.0,CCTG,2024-01-18,0.467532,...,0.311688,0.316883,0.279221,0.290909,0.206494,0.225974,0.232468,0.196104,0.198701,0.228571
71,14.039482,14.220914,13.832130,13.987644,1044800,0.0,0.0,PSBD,2024-01-18,1.016059,...,1.062494,1.060584,1.061857,1.081292,1.066809,1.074739,1.096368,1.063983,1.060609,1.095498
72,3.990000,4.000000,3.260000,3.610000,711400,0.0,0.0,SYNX,2024-01-12,0.994460,...,1.002770,0.664820,0.753463,0.761773,0.844875,0.844875,0.822715,0.767313,0.825485,1.177285
73,23.820000,24.670000,23.500000,24.150000,426100,0.0,0.0,SDHC,2024-01-16,1.117598,...,1.104348,1.209938,1.036025,1.130849,1.426915,1.484886,1.548654,1.285714,1.340373,1.024017


In [20]:
# Step 4: Compute descriptive statistics for the resulting DataFrame: Use .describe() or similar to analyze each of the 12 columns:
# Select only the future growth columns
growth_columns = [f'future_growth_{i}m' for i in range(1, 13)]
growth_df = future_df[growth_columns]

# Get statistics
stats = growth_df.describe()

# Display the result
print(stats)

       future_growth_1m  future_growth_2m  future_growth_3m  future_growth_4m  \
count      22046.000000      20471.000000      18896.000000      17321.000000   
mean           1.034867          1.061463          1.071810          1.090387   
std            0.783861          1.052754          1.159023          1.264605   
min            0.020233          0.011224          0.008863          0.013852   
25%            0.853948          0.764254          0.686667          0.620097   
50%            0.993748          0.971129          0.946786          0.925816   
75%            1.080117          1.098730          1.109856          1.118937   
max           33.944955         23.888887         21.764706         16.454082   

       future_growth_5m  future_growth_6m  future_growth_7m  future_growth_8m  \
count      15746.000000      14171.000000      12596.000000      11021.000000   
mean           1.106785          1.120480          1.120546          1.109408   
std            1.353261    

In [21]:
# Step 5: Determine the best holding period: Find the number of months (1 to 12) where the average (mean) future growth is maximal.
# This optimal month shows an uplift of >1% compared to all others. Still, the average return remains less than 1 (i.e., expected return is less than doubling your investment).
max_mean_month = stats.loc['mean'].idxmax()
max_mean_value = stats.loc['mean'].max()
print(f"The best holding period to maximize growth is", max_mean_month[-2], "months. With a mean growth of", round(max_mean_value, 3))

The best holding period to maximize growth is 7 months. With a mean growth of 1.121


### Question 4: [Strategy] Simple RSI-Based Trading Strategy


**What is the total profit (in $thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?**


---

#### Goal:
Apply a simple rule-based trading strategy using the **Relative Strength Index (RSI)** technical indicator to identify oversold signals and calculate profits.

In [24]:
# Run the full notebook from Lecture 2 (33 stocks)
# Step 1 & 2: Download precomputed data using this snippet:
import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=cde3525b-0a07-46cd-8f82-60af92c7eff5
To: d:\vapi_phase\Finance\Repo\stock-markets-analytics-zoomcamp\02-dataframe-analysis\data.parquet
100%|██████████| 130M/130M [03:10<00:00, 684kB/s]  


### [EXPLORATORY] Question 5: Finding Your Strategy for IPOs

You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?


#### Steps:

1. **Run the full notebook from Lecture 2 (33 stocks)**  
   - Ensure you can generate the merged DataFrame containing:  
     - OHLCV data  
     - Technical indicators  
     - Macro indicators  
   - Focus on getting **RSI** computed using **Code Snippets 8 and 9**.  
   - This process is essential and will help during the capstone project.

2. ⚠️ **IMPORTANT** Please use this file to solve the Home Assignment (**all next steps**)
 
   Download precomputed data using this snippet:

   ```python
   import gdown
   import pandas as pd

   file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
   gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
   df = pd.read_parquet("data.parquet", engine="pyarrow")

3. **RSI Strategy Setup:**  
   - RSI is already available in the dataset as a field.  
   - The threshold for **oversold** is defined as `RSI < 25`.

4. **Filter the dataset by RSI and date:**  
   ```python
   rsi_threshold = 25
   selected_df = df[
       (df['rsi'] < rsi_threshold) &
       (df['Date'] >= '2000-01-01') &
       (df['Date'] <= '2025-06-01')
   ]
5. **Calculate Net Profit Over 25 Years:**  
   - Total number of trades: **1568**  
   - For each trade, you invest **$1000**  
   - Use the 30-day forward return (`growth_future_30d`) to compute net earnings:  
     ```python
     net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
     ```

   - **Final Answer:**  
     What is the **net income in $K** (i.e., in thousands of dollars) that could be earned using this RSI-based oversold strategy from 2000–2025?

### Q5. [Exploratory, Optional] Predicting a Positive-Return IPO

Most of the strategies for investing in IPOs deliver **negative average and median returns** (and even 75% quantiles).

**Question:**  
How would you change the strategy if you want to **increase the profitability**?

> This is an open-ended brainstorming question — propose ideas for identifying IPOs with positive future returns or building a more effective trading strategy.